# M3 Agentic AI - 邮件助理工作流

## 1. 引言

### 1.1 实验概览

在本模块的前面视频中，Andrew 展示了一个可以使用多种工具执行复杂任务的日历助理示例。在本次无评分实验中，你将探索一个类似的例子——**邮件助理智能体**。 

该智能体工作流可执行与邮件管理相关的多项任务，包括发送邮件、查找特定发件人的邮件以及删除邮件。你将给予自然语言指令——例如“查看老板的未读邮件”或“删除 Happy Hour 的邮件”——并观察它如何选择合适的工具为你完成任务。

<img src="lab_overview.png" alt="日历助理示例" width="700"/>

### 🎯 1.2 学习目标
在本实验结束时，你将能够使用 AISuite 将 **LLM 与工具连接**，提供自然语言指令，并观察智能体如何选择、执行与验证诸如搜索、发送、删除邮件等多步任务。


## 2. 初始化环境与客户端

与之前的实验一样，你现在将完成环境设置。  

In [1]:
# ================================
# 导入
# ================================

# --- 第三方 ---
from dotenv import load_dotenv
import aisuite as ai
import json

# --- 本地 / 项目 ---
import utils
import display_functions
import email_tools



# ================================
# 环境与客户端
# ================================
load_dotenv()          # 从 .env 加载环境变量
client = ai.Client()   # 初始化 AISuite 客户端


## 3. 模拟邮件服务

### 3.1 组件
本实验使用一个**模拟的邮件后端**来仿真真实世界的邮件交互。
你可以把它当作你的沙盒邮件收件箱：其中预置了消息，方便你在无需发送真实邮件的情况下进行练习。  

无需从零构建该后端；你将使用工具直接与之交互。其底层组件包括：

| 层级                    | 作用                             |
|-------------------------|---------------------------------|
| **FastAPI**             | 提供 REST 接口                   |
| **SQLite + SQLAlchemy** | 本地存储并查询邮件               |
| **Pydantic**            | 保证输入与输出有效性             |
| **AISuite 工具**        | 连接 LLM 与服务的桥梁            |


### 3.2 接口
该服务提供若干路由以模拟常见的邮件操作。  
接下来，这些路由将被包装为工具，助理可自动调用：

- `POST /send` → 发送新邮件  
- `GET /emails` → 列出所有邮件  
- `GET /emails/unread` → 仅显示未读邮件  
- `GET /emails/{id}` → 按 ID 获取邮件  
- `GET /emails/search?q=...` → 按关键词搜索邮件  
- `GET /emails/filter` → 按收件人或日期范围筛选  
- `PATCH /emails/{id}/read` → 将邮件标记为已读  
- `PATCH /emails/{id}/unread` → 将邮件标记为未读  
- `DELETE /emails/{id}` → 按 ID 删除邮件  
- `GET /reset_database` → 重置到初始状态（用于测试）  


> 💡 **关键想法：** 这些接口相当于“收件箱控制”。  
> 在接下来的步骤中，你将把它们暴露为 LLM 可调用的 Python 函数（工具）——将原始路由转化为智能体动作。

### 3.3 接口测试辅助函数
在交给 LLM 控制之前，你将先**自行测试后端**。  
`utils.test_*` 函数是针对 API 接口的快速封装，让你无需编写原始 HTTP 请求即可尝试**发送、列表、搜索、过滤、标记与删除**等操作。

_**注意：** 可在顶部菜单通过 File > Open 找到 utils.py 文件_

每个辅助函数都有清晰、易懂的名称。接下来你可以在下一个单元中取消注释想要运行的函数，查看其输出，并确认邮件服务表现符合预期。  

例如，你可以测试：  
- 发送测试邮件  
- 通过 ID 获取邮件  
- 列出所有消息  
- 将邮件标记为已读或未读  
- 删除邮件  

在交付控制权给智能体之前，这是你的**健全性检查**。它让你确信模拟的邮件服务正在运行，并表现得与真实邮件服务相同。

👉 要测试接口，请**取消注释**你想尝试的行，运行单元并立即查看结果。

In [2]:
# 取消注释你想尝试的 'utils.test_*' 行
new_email_id = utils.test_send_email()
_ = utils.test_get_email(new_email_id['id'])
#_ = utils.test_list_emails()
#_ = utils.test_filter_emails(recipient="test@example.com")
#_ = utils.test_search_emails("lunch")
#_ = utils.test_unread_emails()
#_ = utils.test_mark_read(new_email_id['id'])
#_ = utils.test_mark_unread(new_email_id['id'])
#_ = utils.test_delete_email(new_email_id['id'])
#_ = utils.reset_database()


## 4. 邮件助理的工具层

### 4.1 为什么需要工具？
既然接口已正常工作，下一步是将它们以 Python 函数形式暴露给 LLM，称为**工具（tools）**。每个工具都会封装一个 REST 路由，将原始 API 调用转化为智能体可执行的动作——如列表、读取、搜索、发送、删除或切换已读状态。

> 可以把工具看作智能体的**执行器**：你给出自然语言指令（例如“检查老板的未读邮件并发送礼貌的回复”），模型会选择**调用哪些工具**以及**以什么顺序**完成任务。

### 4.2 设计提示
- 保持工具的 **docstring** 简短、使用祈使句并聚焦具体动作。  
- 返回 **一致且紧凑的 JSON**，便于模型串联使用结果。  
- 倾向于 **每个工具只承担一个清晰职责**（单一路由、单一效果）。

### 4.3 可用工具
| 工具函数                          | 动作                                                                 |
|------------------------------------|------------------------------------------------------------------------|
| `list_all_emails()`                | 获取所有邮件（按时间倒序）                                             |
| `list_unread_emails()`             | 仅检索未读邮件                                                         |
| `search_emails(query)`             | 按主题、正文或发件人关键词搜索                                         |
| `filter_emails(...)`               | 按收件人与/或时间范围筛选                                              |
| `get_email(email_id)`              | 通过 ID 获取指定邮件                                                   |
| `mark_email_as_read(id)`           | 将邮件标记为已读                                                      |
| `mark_email_as_unread(id)`         | 将邮件标记为未读                                                      |
| `send_email(...)`                  | 发送一封新的（模拟）邮件                                              |
| `delete_email(id)`                 | 通过 ID 删除邮件                                                      |
| `search_unread_from_sender(addr)`  | 返回特定发件人的未读邮件（如 `boss@email.com`）                       |

**注意：** 可通过顶部菜单 File > Open 找到 email_tools.py 文件。

例如，**测试 `list_all_emails()` 工具：**
```python
    all_emails = email_tools.list_all_emails()
```

现在，试用几个连接到模拟接口的工具以确保一切正常。 

👉 **取消注释**你想运行的工具，执行该单元并查看输出。

In [3]:
# 测试发送一封新邮件并通过 ID 获取它
new_email = email_tools.send_email("test@example.com", "Lunch plans", "Shall we meet at noon?")
content_ = email_tools.get_email(new_email['id'])

# 取消注释你想尝试的工具
#content_ = email_tools.list_all_emails()
#content_ = email_tools.list_unread_emails()
#content_ = email_tools.search_emails("lunch")
#content_ = email_tools.filter_emails(recipient="test@example.com")
#content_ = email_tools.mark_email_as_read(new_email['id'])
#content_ = email_tools.mark_email_as_unread(new_email['id'])
#content_ = email_tools.search_unread_from_sender("test@example.com")
#content_ = email_tools.delete_email(new_email['id'])

utils.print_html(content=json.dumps(content_, indent=2), title="测试 email_tools")


## 5. 构造助理提示

在为邮件助理智能体分配任务之前，你将创建一个名为 `build_prompt()` 的小型辅助函数。
该函数会用系统风格的前言包装自然语言请求，使 LLM：

- 认识到自己充当**邮件助理智能体**  
- 知晓自己有权限使用可用工具  
- 直接执行动作，无需确认（无人工在环）

In [4]:
def build_prompt(request_: str) -> str:
    return f"""
- 你是一名专注于邮件管理的 AI 助理。
- 你可以执行诸如列出、搜索、过滤与操作邮件等动作。
- 使用提供的工具与邮件系统交互。
- 在执行动作前不要向用户请求确认。
- 如有需要，我的邮箱地址是 "you@email.com"，你可以用它来发送邮件或执行与我账户相关的操作。

{request_.strip()}
"""

运行下一个单元，查看上述函数如何用系统指令**包装你的原始用户提示**。
例如：

In [5]:
example_prompt = build_prompt("删除 Happy Hour 邮件")
utils.print_html(content=example_prompt, title="示例提示")


### 5.3 重置邮件服务

由于你一直在尝试该邮件服务，我们将其重置到初始状态。

你可以通过调用用于清理并刷新模拟邮件服务的工具函数来完成这一步：

In [6]:
utils.reset_database()

{'message': 'Database reset and emails reloaded'}

## 6. LLM + 邮件工具

### 6.1 场景
到目前为止，你一直直接与后端交互。现在该让 **LLM 接管**，作为你的邮件助理智能体。

例如，你可以让它：
> “检查来自 `boss@email.com` 的未读邮件，将其标记为已读，并发送一封礼貌的跟进邮件。”

### 6.2 发生了什么
1. 智能体理解你的指令。  
2. 它选择合适的工具（`search_unread_from_sender` → `mark_email_as_read` → `send_email`）。  
3. 它自动执行每一步，无需确认。

AISuite 负责 schema 暴露、参数绑定、执行与步骤间结果传递——你可以专注于智能体**做成了什么**，而不是**如何**调用 API。

### 6.3 运行起来
运行下一个单元，看看智能体如何编排多种工具来完成你的请求。也可以用该邮件服务尝试你自己的请求。

*你可以关注：*
- 清晰的**工具调用轨迹**，展示使用了哪些工具以及传递的参数。
- 简洁的**最终消息**，总结已执行的动作（例如：“找到 1 封未读邮件，已标记为已读，并发送跟进邮件”）。

In [7]:
# 尝试你自己的请求
prompt_ = build_prompt("检查来自 boss@email.com 的未读邮件，将其标记为已读，并发送一封礼貌的跟进邮件。")

response = client.chat.completions.create(
    model="openai:gpt-4.1", # LLM
    messages=[{"role": "user", "content": (
        prompt_
    )}],
    tools=[ # LLM 可访问的工具列表
        email_tools.search_unread_from_sender,
        email_tools.list_unread_emails,
        email_tools.search_emails,
        email_tools.get_email,
        email_tools.mark_email_as_read,
        email_tools.send_email
    ],
    max_turns=5,
)

display_functions.pretty_print_chat_completion(response)



### 6.4 缺失的工具：`delete_email`

当所需工具不可用时会发生什么？  

例如，尝试这个请求：  
> “请删除 alice@work.com 的邮件。”

由于 `delete_email` 工具尚未注册，LLM 仍会尝试响应，但无法完成删除操作。   

<div style="background-color: #ffe4e1; padding: 12px; border-radius: 6px; color: black;">
  <h4>🔍 关键洞察</h4>
  <p style="margin: 0;">
    这强调了一个重要事实：<b>你提供哪些工具将直接决定智能体能做什么。</b>
  </p>
</div>


**可用工具**

| 工具函数                           | 动作                                                                  |
|------------------------------------|------------------------------------------------------------------------|
| `list_all_emails()`                | 获取所有邮件（最新优先）                                              |
| `list_unread_emails()`             | 仅检索未读邮件                                                         |
| `search_emails(query)`             | 按主题、正文或发件人关键词搜索                                         |
| `filter_emails(...)`               | 按收件人与/或日期范围筛选                                              |
| `get_email(email_id)`              | 按 ID 获取指定邮件                                                     |
| `mark_email_as_read(id)`           | 将邮件标记为已读                                                      |
| `mark_email_as_unread(id)`         | 将邮件标记为未读                                                      |
| `send_email(...)`                  | 发送一封新的（模拟）邮件                                              |
| `delete_email(id)`                 | 按 ID 删除邮件                                                        |
| `search_unread_from_sender(addr)`  | 返回特定发件人的未读邮件（如 `boss@email.com`）                       |





In [8]:
# 尝试一个可能会调用不可用工具的请求
prompt_ = build_prompt("删除 alice@work.com 的邮件")

response = client.chat.completions.create(
    model="openai:o4-mini",
    messages=[{"role": "user", "content": (
        prompt_
    )}],
    tools=[ # LLM 可访问的工具列表
        email_tools.search_unread_from_sender,
        email_tools.list_unread_emails,
        email_tools.search_emails,
        email_tools.get_email,
        email_tools.mark_email_as_read,
        email_tools.send_email
    ],
    max_turns=5
)

display_functions.pretty_print_chat_completion(response)


#### 6.4.1 启用 `delete_email` 后再试一次

当所需工具不可用时会发生什么？

在上一步中，你为 LLM 提供了如下工具列表：
```python
    tools=[
        email_tools.search_unread_from_sender,
        email_tools.list_unread_emails,
        email_tools.search_emails,
        email_tools.get_email,
        email_tools.mark_email_as_read,
        email_tools.send_email
    ]
```

由于该列表中的可用工具有限，智能体无法删除邮件，因为 `delete_email` 工具尚不可用。

现在将 `delete_email` 加入工具列表并重新运行该单元。这一次，智能体将具备完成任务所需的一切。  

> 提示：留意调用序列——在找到目标邮件后，智能体应选择 `delete_email` 来完成删除操作。


In [9]:
prompt_ = build_prompt("删除 alice@work.com 的邮件")

response = client.chat.completions.create(
    model="openai:o4-mini",
    messages=[{"role": "user", "content": (
        prompt_
    )}],
    tools=[
        email_tools.search_unread_from_sender,
        email_tools.list_unread_emails,
        email_tools.search_emails,
        email_tools.get_email,
        email_tools.mark_email_as_read,
        email_tools.send_email,
        email_tools.delete_email
    ],
    max_turns=5
)

display_functions.pretty_print_chat_completion(response)


### 6.5 定向操作：删除“Happy Hour”邮件

测试收件箱预置了一封主题为 **“Happy Hour”** 的邮件。你的任务是指导智能体定位该邮件并将其删除。

供参考，以下是模拟数据集中该邮件的条目：

```json
{
  "id": 1,
  "sender": "eric@work.com",
  "recipient": "you@email.com",
  "subject": "Happy Hour",
  "body": "We're planning drinks this Friday!",
  "timestamp": "2025-06-13T04:48:59.096908",
  "read": false
}
````
运行下一个单元，观察智能体如何搜索该邮件并将其从收件箱中删除。

In [10]:
prompt_ = build_prompt("删除 Happy Hour 邮件")

response = client.chat.completions.create(
    model="openai:o4-mini",
    messages=[{"role": "user", "content": (
        prompt_
    )}],
    tools=[
        email_tools.search_unread_from_sender,
        email_tools.list_unread_emails,
        email_tools.search_emails,
        email_tools.get_email,
        email_tools.mark_email_as_read,
        email_tools.send_email,
        email_tools.delete_email
    ],
    max_turns=5
)

display_functions.pretty_print_chat_completion(response)

## 7. 关键收获

- 在本次无评分实验中，**你探索了** LLM 邮件助理如何通过工具调用与模拟邮件服务交互。
- **工具调用**让 LLM 超越文本生成——能够调用函数（工具）并完成多步任务。
- 可用工具的集合决定了智能体能与不能做的事（例如没有 `delete_email` 就无法删除消息）。
- 清晰的 docstring 与一致的行为有助于 LLM 在每一步选择合适的工具。
- AISuite 管理交互层：将 Python 函数暴露为工具、接收参数、发起 API 请求并返回结果。
- 观察完整工作流（从提示 → 工具调用 → 输出 → 最终响应）是理解并提升智能体推理与行动的关键。

<div style="border:1px solid #22c55e; border-left:6px solid #16a34a; background:#dcfce7; border-radius:6px; padding:14px 16px; color:#064e3b; font-family:system-ui,-apple-system,Segoe UI,Roboto,Ubuntu,Cantarell,Noto Sans,sans-serif;">

🎉 <strong>恭喜！</strong>

你刚刚将一个由 LLM 驱动的<em>邮件助理智能体</em>从自然语言请求引导到在模拟邮件服务上的具体行动。你看到了工具如何将纯文本请求转化为可靠的操作——列表、搜索、标记已读、发送与删除——而无需直接编写原始 HTTP 请求。

你也了解了所暴露的工具定义了智能体的真实能力：缺少某工具时智能体能推理但不能行动；具备时则能串联步骤、传递参数并准确完成你的需求。整个过程中，AISuite 负责底层管道：schema 暴露、参数绑定、执行与结果传递——让你专注于“要做什么”，而非“如何接线”。

掌握该工作流后，你已准备好设计以任务为中心的智能体：安全透明地行动、解释所做之事，并从简单提示扩展到可靠的多步自动化，贯穿你的自有服务。🌟
</div>